In [2]:
# logging and decorators
import logging as log

# tensorflow modules
import tensorflow as tf
import tensorflow_text as tf_text
from tensorflow.python.keras import layers

# necessary for visualization and user input
import ipywidgets as widgets
from ipywidgets import interact_manual, interactive, interact, VBox, HTML
from IPython.display import display, clear_output

# modules from backend
from interactive_inference_backend import ModelLoader, StoryTokenizer, WordComplete, VisualWrapper, positional_encoding
from interactive_inference_backend import reserved_tokens, vocab_path

In [3]:
model = ModelLoader(StoryTokenizer(reserved_tokens, vocab_path),
                            d_model=512,
                            n_stacks=2,
                            h_att=4,
                            load_model=True,
                            model_load_path="model_N2_h4_d512_t20230831-134344")

# Interaktive Erklärung der Transformerarchitektur
## Einleitung
### Kurzübersicht Transformer

Transformer-Modelle sind eine von Vaswani et al. [1] vorgeschlagene Architektur für das Modellieren von sequentiellen Daten. Im Gegensatz zu den vorher genutzten Architekturen wie RNNs [2, 3] oder CNNs [4] ermöglichen Transformer allerdings das parallele Verarbeiten der sequentiellen Daten während des Trainings und ermöglichen dadurch mit erheblich reduzierter Trainingszeit und -rechenkapazität sequentielle Daten zu verarbeiten [Cite comparisson].

Um diese parallele Verarbeitung zu erreichen nutzen Transformermodelle Aufmerksamkeitsblöcke. Aufmerksamkeitsblöcke sind trainierbare Matrizen, die von einem Eingabevektor auf einen Ausgabevektor projizieren. Also eine sequentielle Datenstruktur in Vektorform in eine ebenfalls sequentielle Datenstruktur in Vektorform verarbeiten, indem eine Matrixmultiplikation auf dein Eingabevektor angewandt wird. Wie das funktioniert wird in [einer eigenen Sektion behandelt](#aufmerksamkeit) gezeigt.
Diese Matrizen projizieren also im Idealfall von jedem Eintrag im Eingabevektor genau diejenige Information auf einen Eintrag im Ausgabevektor, die an der jeweiligen Stelle die Ausgabe beeinflussen soll. Da hierbei die Ausgabe i nicht von der Ausgabe i-1 abhängt, können wir parallel die komplette Ausgabe zu jeder Eingabe erzeugen und mit einem Gradient-Descent [5] Verfahren unser Modell trainieren
Da wir während der Inferenz (Erklärung was Inferenz ist) von sequentiellen Daten aber meist nur die vorhergehende Daten nutzen möchten benutzen Transformer Architekturen eine Maske, die nach der Anwendung der Matrixmultiplikation alle Informationen, die nachfolgende Daten liefern herausfiltern, sodass schon während dem Training nur die Verbindung zwischen der aktuellen Position und den vorhergehenden gelernt wird.
Durch wiederholen von Aufmerksamkeitsblöcken und einem nachfolgenden Feed-Forward Netzwerk entsteht so eine Architektur zur Verarbeitung sequenzieller Daten, die verglichen mit vorherigen Architekturen weniger Rechenkapazität benötigt und trotzdem eine wesentlich reduzierte Trainigszeit aufweist. [1, 6]

In [4]:
# Hier eine grafische Darstellung von Eingabe- zu Ausgabevektor darstellen. Wie werden sequentielle Daten verarbeitet und wie macht das ein Transformer



### Ziel dieses interaktiven Artikels

Ziel dieses interaktiven Artikels soll es sein die Architektur die [1] beschreiben in ihren einzelnen Komponenten darzustellen. Der Fokus liegt hierbei darauf die Prozesse, die während der Verarbeitung sequentieller Daten stattfinden, grafisch durch interaktive Anwendungen darzustellen der Nutzer den Einfluss unterschiedlicher Architekturbausteine auf verschiedene Eingaben anschaulich klar wird. Ziel ist es, die verschiedenen Bausteine so zu erklären, dass einem möglichen Anwender die Implementation einer Tranformer-Architektur, durch ein Verständnis des Nutzens der einzelnen Architekturbausteine, erleichtert wird.

Der Artikel soll dazu dienen eine Implementierung ohne einschlägiges Vorwissen, z.B. im Kontext von KMUs die sich bisher noch nicht mit der Modellierung sequentieller Daten beschäftigt haben, zu ermöglichen und Programmierer in der Designentscheidungen in der Transformerarchitektur zu unterstützen.
[1] und viele der auf ihrer Architektur aufbauenden wissenschaftlichen Arbeiten [7, 8], Erklärartikel oder -videos [9, 10] beschränken sich, wenn überhaupt, darauf den Aufmerksamkeitsmechanismus ausführlich darzustellen. Dabei werden die Designentscheidungen für trainingsrelevante Elemente wie Dropout [11], Residuale Verbindungen [12] sowie die Beschreibung bereits etablierter Methoden wie Byte-Pair Encoding [13] als Einbettung oder Log-Softmax [5] vernachlässigt. Diese Elemente sollen hier aber ebenfalls dargestellt werden.

### Architekturübersicht

In <a href="#fig:input">Abbildung 1</a> ist eine vollständige Darstellung aller Architekturelemente zu finden, die Teil der Transformerarchitektur sind. Dabei unterscheiden wir vier verschiedene Elemente. Prozesse und Methoden werden in Schwarz dargestellt. Die in diesen Prozessen generierte Daten werden in Blau dargestellt. Trainierbare Parameter werden in Gelb dargestellt. Hyperparameter, also festzulegende Eingabeparameter für das Modell werde in Grau dargestellt. Ebenfalls in Grau werden Eingabe- und Ausgabedaten dargestellt.
Unsere Abbildung ist insofern komplett, als sie jeglichen Weg zeigen, den Daten durch das Modell nehmen können. Eine Darstellung, in der Form eines Encode-Decoder Netzwerks [14], wie sie [1] nutzen findet sich in Abbildung 2.

<figure id="fig:input" style="height: 700px;">
  <img src="./img/tf_arch_full.jpg" style="height: 1400px;" alt="Eingabepipeline mit Tokenizer, Embedding und Positional Encoding."/>
  <figcaption>Abbildung 1: Transformerarchitektur</figcaption>
</figure>
<br>


#### Encoder-Decoder

Eine Encoder-Decoder Architektur besteht aus zwei voneinander getrennten neuronalen Netzen dem Encoder und dem Decoder, die aber zusammen trainiert werden. Diese Architektur wurde von [14] vorgeschlagen und dann vor allem im Kontext von Neural Machine Translation (NMT) verwendet. Der Vorteil von Encoder-Decoder liegt darin, dass man sowohl den Encoder als auch den Decoder ersetzen kann. Im Idealfall könnte man im Bereich von NMT pro Sprache einen Encoder und einen Decoder trainieren, um dann beliebige Übersetzungen zwischen allen Sprachen zu generieren. Da auch [15], die zuerst Aufmerksamkeitsmechanismen für Rekurrente Neuronale Netze (RNN) vorgeschlagen haben, eine Encoder-Decoder Architektur verwenden, verwundert es nicht, dass auch [1] eine solche Architektur wählten. Insbesondere da [1] Transformer ebenfalls für NMT benutzen.
Transformer zur Textvervollständigung können auch ausschließlich auf Decodern beruhen [16]. In unserem Beispiel wird der Encoder zum enkodieren der Eingabe verwendet, um dann im Decoders die Eingabe als Anfang der Ausgabe zu verwenden. In Abbildung 2 sieht man wie Encoder und Decoder zusammenarbeiten. Der Decoder benutzt Selbstaufmerksamkeit, um seinen Input zu verarbeiten, nutzt dann Kreuz-Aufmerksamkeit, für den nächsten Schritt, um final erneut Selbstaufmerksamkeit zu benutzen, wohingegen der Encoder ausschließlich über Selbstaufmerksamkeit funktioniert. Näheres siehe im Kapitel [Aufmerksamkeit](#aufmerksamkeit).

In [5]:
tokenizer = StoryTokenizer(reserved_tokens, vocab_path)
transformer = model.model

input_widget_enc_dec = widgets.Text(
    value='Encoder-Decoder Test',
    description='Ihre Eingabe:',
    continuous_update=False,  # updates value only when you finish typing or hit "Enter"
    layout = widgets.Layout(width='auto', margin='0px 0px 10px 0px')
)

button_widget_enc = widgets.Button(description='Enkodiere die Eingabe',
                               layout = widgets.Layout(width='auto'))
button_widget_dec = widgets.Button(description='Dekodiere die Eingabe',
                               layout = widgets.Layout(width='auto'))

output_widget_enc = widgets.Output(layout = widgets.Layout(width='auto'))
output_widget_dec = widgets.Output(layout = widgets.Layout(width='auto'))


def encode():
    tensor_input = tf.convert_to_tensor(input_widget_enc_dec.value)            # Umwandelung des Textinputs in ein TensorFlow-Tensor 
    if len(tensor_input.shape) == 0:                                           # Überprüft, ob der Eingabetensor im korrekten Format ist
      tensor_input = tensor_input[tf.newaxis]                                  # Falls nicht, wird eine Dimension hinzufügt 
    
    tokenized_input = tokenizer.tokenize(tensor_input).to_tensor()             # Umwandlung des Textinputs in Tokens und anschließend in einen Tensor
    input_without_eos = tokenized_input[:, :-1]
    token_input = tokenizer.detokenize(input_without_eos)
    lookup = tokenizer.lookup(input_without_eos)
    lookup = [item.decode('utf-8') for sublist in lookup.numpy() for item in sublist]

    string_value = token_input.numpy()[0].decode('utf-8')
    
    context = transformer.encode(input_without_eos, None)                      # Kodierung des Inputsatzes von (Transformer-Modell)
    return context, string_value, lookup


def decode(): 
    tensor_input = tf.convert_to_tensor(input_widget_enc_dec.value)            # Umwandelung des Textinputs in ein TensorFlow-Tensor 
    output_array = tf.TensorArray(dtype=tf.int64, size=0, dynamic_size=True)   # Erstellung eines leeren TensorArrays für die spätere Ausgabe
    if len(tensor_input.shape) == 0:                                           # wie bei der Encodierung
      tensor_input = tensor_input[tf.newaxis]                                  # wie bei der Encodierung

    tokenized_input = tokenizer.tokenize(tensor_input).to_tensor()             # wie bei der Encodierung
    input_without_eos = tokenized_input[:, :-1]
    
    token_input = tokenizer.detokenize(input_without_eos)
    string_value = token_input.numpy()[0].decode('utf-8')                      
    context = transformer.encode(input_without_eos, None)                     
    lookup = tokenizer.lookup(input_without_eos)
    lookup = [item.decode('utf-8') for sublist in lookup.numpy() for item in sublist]

                              
    for i, value in enumerate(tokenized_input[0][:-1]):                        # Schleife durch jedes Token des Satzes
      output_array = output_array.write(i, value)                              # Speichern des Tokens im Output array

    dec_input = output_array.concat()[tf.newaxis]                              # Output Array wird zu einem einzigen Tensor konkateniert 
                                                                               # und anschließend um eine zusätzliche Dimension erweitert

    dec_out = transformer.decode(context, None, dec_input, None)               # Decoder des Transformer-Modells wird verwendet, um den dec_input-Tensor 
                                                                               # unter Verwendung des zuvor berechneten Kontexts zu decodieren.

    return dec_out, string_value, lookup





def on_button_click_enc(b):
  with output_widget_enc:
    output_widget_enc.clear_output()  # clear the previous output
    context, tokens, lookup = encode()
    print('Wörter: ', tokens)
    print('Tokens: ', lookup)
    print('\n')
    #VisualWrapper.display_text('Beispieltext')
    VisualWrapper.color_bar(context) 


def on_button_click_dec(b):
  with output_widget_dec:
    output_widget_dec.clear_output()
    dec_out, tokens, lookup = decode()
    print('Wörter: ', tokens)
    print('Tokens: ', lookup)
    print('\n')
    #VisualWrapper.display_text('Beispieltext')
    VisualWrapper.color_bar(dec_out)



  

button_widget_enc.on_click(on_button_click_enc)
button_widget_dec.on_click(on_button_click_dec)

widgets.VBox([input_widget_enc_dec, widgets.HBox([widgets.VBox([button_widget_enc, output_widget_enc]), widgets.VBox([button_widget_dec, output_widget_dec])])])
#display(input_widget_enc_dec, button_widget_enc, output_widget_enc, button_widget_dec, output_widget_dec)


#print('tok_out', tokenized_input)
#print('enc_out', context)
#print("dec_out", dec_out)

#### Erklärung des Beispiels
Über der Grafik sind die verarbeiteten Tokens zu sehen. In der Grafik wird die Position der Tokens wird durch die y-Achse angezeigt und entspricht der Reihenfolge der Tokens im Beispielsatz.  Dabei werden Leerzeichen weggelassen. Das Model erkennt unterschiedliche Wörter mit Hilfe der "##" Zeichen. Durch diese Zeichen sieht das Modell, dass ein Token mit "##" Zeichenkette zu dem vorherigen Token gehört wie im Beispiel bei dem Wort "Endcoder", welches aus den Tokens 'e', '##n', '##co', '##der' besteht. Weitere Informationen in Abschnitt [Byte-Pair Encoding](#byte-pair-encoding).

Schaut man sich die Wörter vor der Tokenisierung an, sieht man das alle Wörter nun kleingeschrieben werden. Somit werden groß- und kleingeschriebene Formen des Wortes zusammengefasst. Dadurch wird die Anzahl des Vokabulars reduziert. Linguistische und grammatische Informationen, die aufgrund der Zusammenführung verloren zu gehen scheinen, werden durch die Position und den Kontext des Wortes im Satz erhalten (z.B. Substantivierungen). Ebenso wird das Modell dazu verleitet, mehr Acht auf die Position des Wortes im Satz zu geben.  Der Tokenizer fügt außerdem ein Start- und Endtoken hinzu. Das Endtoken wird im Modell nicht weiterverwendet, während das Starttoken die Position 0 in der weiteren Verarbeitung und Visualisierung einnimmt.

Die x-Achse repräsentiert die Tiefe der Token bzw. die Position der einzelnen Werte des Vektors, der die Tokens darstellt. In diesem Fall beträgt die Tiefe 512, was bedeutet, dass die Wörter durch 512 verschiedene Werte dargestellt werden. Diese Vektoren sind jedoch nicht zufällig, sondern weisen untereinander Abhängigkeiten auf. Das heißt, ähnliche Wörter haben z.B. ähnliche Vektoren. Darüber hinaus sind in diesen Vektoren auch die Abhängigkeiten der Wörter im Satz enthalten, was als sogenannt [Positional Encoding](#positional-encoding) bezeichnet wird.  
.  




#### Architekturblöcke

Prinzipiell lassen sich Transformer in mehrere Blöcke einteilen.

1. Eingabeblock

Die Eingabepipeline verarbeitet die Eingabedaten in eine Form die für die Matrixtransformation im Aufmerksamkeitsblock genutzt werden kann. Diese unterscheidet sich für verschiedene Datentypen. In unserem Beispiel nutzen wir Textdaten, die wir durch Tokenisierung [17] und ein Embedding [18] in Tensoren verwandeln.

2. Aufmerksamkeitsblock

Der Aufmerksamkeitsblock verarbeitet Daten in Tensorform und liefert somit eine Abbildung von den Eingabetensoren auf die Ausgabetensoren, die jeweils von den Eingabe- und Ausgabeblöcken interpretiert wird.
Man kann verschiedene Modulvarianten innerhalb eines Aufmerksamkeitsblocks unterscheiden. Aufmerksamkeitsmodule erhalten als Eingabedaten immer drei Tensoren. Diese werden Query, Key und Value genannt. Aus diesen berechnet ein Aufmerksamkeitsmodul eine Ausgabe.
Typischerweise werden Aufmerksamkeitsmodule dabei dadurch unterschieden aus welcher Quelle Query, Key und Value stammen.
Es gibt 

- Self-Attention bei der Query, Key und Target alle aus einer Quelle stammen und
- Source-Attention (Hier ist der Query aus einer anderen Quelle als Key und Value, z.B. wenn der Query die Ausgabe eines Encoders ist, während Target die Ausgabe eines Decoderblocks ist).

Desweiteren kann man nach Art des Maskings unterscheiden. Masking wird verwendet, um das Aufmerksamkeitsmodul daran zu hindern aus einem Teil der Ausgabedaten zu lernen. In Transformern gibt es 

- Subsequent Masking, das ein Decoderaufmerksamkeitsblock daran hindert für die Vorhersage der Position i Daten aus den Positionen i+j nutzt, und
- Padding Masking, das verhindert, dass Fehler verhindert, die aufgrund dessen entstehen, dass die Eingabe eines Transformers, unabhängig vom Inhalt, immer die selbe Anzahl an Tokens besitzen muss.

3. Ausgabeblock

Die Ausgabepipeline interpretiert die Ausgaben des Aufmerksamkeitsmoduls, sodass sie in eine für menschlichen Gebrauch nützlichen Form vorliegen (typischerweise z.B. Textdaten, Bilddaten, etc.). Dafür wird in Transfomern eine Log-Softmax Funktion genutzt, die auf die Tensorausgabe des letzten Aufmerksamkeitsblockes angewandt wird.

In <a href="#fig:input">Abbildung 1</a> entspricht das Aufmerksamkeitsmodul den Prozessen innerhalb der grauen Umrandung, während die Eingabe- und Ausgabepipeline darunter bzw. darüber zu finden sind.

## Input

Der erste Teil eines Transformermodells besteht aus der Eingabepipeline. Diese verarbeitet die Eingabe in das Modells, z.B. die Texteingabe eines Nutzers und bereitet sie darauf vor durch wiederholte Anwendung der Aufmerksamkeitsmodule in einem Transformermodell verarbeitet zu werden. Die Aufmerksamkeitsmodule arbeiten über eine Aufmerksamkeitsmatrix, die aus der jeweiligen Eingabe eine Ausgabe berechnet. Wir müssen also aus einer Eingabe in Textform eine Vektorrepräsentation erzeugen, die alle notwendigen Informationen für ein Modell enthalten, um nützliche Vorhersagen über eine Übersetzung oder eine Textvervollständigung machen zu können.

<a href="#fig:input">Hier</a> sehen sie nochmal den Ausschnitt aus der obigen Grafik, der die Eingabepipeline darstellt. 
Wie zu erkennen ist, wird in der Eingabpipeline während des Training eines Transformermodells ausschließlich die Gewichte für die Einbettung trainiert, alles andere funktioniert deterministisch und ist damit unbeeinflusst vom Trainningsprozess.



<figure id="fig:input" style="height: 700px;">
  <img src="./img/tf_input_pipeline.jpg" style="height: 700px;" alt="Eingabepipeline mit Tokenizer, Embedding und Positional Encoding."/>
  <figcaption>Abbildung 2: Eingabepipeline eines Transformer-Netzwerks</figcaption>
</figure>
<br>


Prinzipiell besteht die Eingabepipeline aus drei Modulen:

1. [Tokenisierung](#tokenisierung),
2. [Einbettung](#einbettung),
3. [Positionale Kodierung](#positional-encoding).

Dies entspricht drei Verarbeitungsschritten:

1. Bei der Tokenisierung, wird der Text mithilfe eines Zeichenalphabets in eine Zahlenkodierung umgewandelt.
2. Bei der Einettung wird diese mithilfe eines trainierbaren Algorithmus in eine Vektordarstellung mit sehr viel mehr Parametern umgewandelt. Diese Einbettung lernt die komplexe Struktur eines Textes so darzustellen, dass sie informativ für die nachfolgenden Module ist. Wie genau das passiert ist dabei aufgrund der stochastischen Natur von Deep Learning Verfahren nicht offen einsehbar.
3. Die Positionelle Kodierung ist einzigartig für Transformermodelle. Im Gegensatz zu RNN, die die Eingabedaten sequentiell präsentiert bekommen [19], bekommen Transformer die Eingabe ohne Informationen zur relativen Position der Tokens. Diese fehlenden Informationen werden in diesem Schritt manuell hinzugefügt.

### Tokenisierung

Bei der Tokenisierung wird der Satz in Textform z.B. "Das ist ein Testsatz." in einen Zahlencode verwandelt. Hierfür kommen verschieden Methoden in Frage. Einer der simpelsten Methoden ist es z.B. jedem Buchstaben eine Zahl zuzuordnen. Das führt allerdings zu einer sehr langen Kodierung.

Die entscheidenden Faktoren für eine gute Kodierung sind
- Vollständigkeit der Kodierung, 
- Länge des kodierten Vektors und 
- Größe des dafür nötigen Vokabulars.

Die Kodierung mit einzelnen Buchstaben ist vollständig (man kann beliebige Zeichenkombinationen kodieren) und besitzt ein kurzes Vokabular (26 für alle Buchstaben plus alle Punktierungs und Sonderzeichen, die im Text vorkommen), allerdings ist die Länge der kodierten Vektoren groß.

Benutzt man ein Vokabular an festgeleten Wörtern, wird die Länge der Kodierung verkürzt, allerdings besteht die Gefahr der Unvollständigkeit. Um das nach Möglichkeit zu verhindern, benötigt man ein Vokabular, dass den Raum der möglichen Wörter vollständig abdeckt. Diese Vokabular müsste daher sehr umfangreich sein.

Die Probleme der obigen Methoden hat dazu geführt, dass sich gemischte Verfahren ergeben haben. Die vorher an einem möglichst großem Korpus trainiert werden. Zum einen Top-Down Verfahren, die von einem aus dem Korpus extrahierten Vokabular ausgehen und lernen unbekannte Worte in Teilworte zu zerlegen, die dann in das Vokabular aufgenommen werden. Zum anderen Bottom-Up Verfahren, die von einem Buchstabenvokabular ausgehen und häufig wiederkehrende Kombinationen in dieses explizit aufnehmen.
Die Transformerarchitektur nach [1] verwendet ein solches Bottom-Up Verfahren namens Byte-Pair Encoding [13], dass sich als effizient erwiesen hat und ein relativ kompaktes Vokabular ermöglicht dabei aber die Länge der Tokenisierung klein hält. 


Den Tokenizer findet man in unserer <a href="#fig:transformer">Abbildung 1</a> ganz unten und ist der erste Schritt, um eine Eingabe zu verarbeiten
<figure id="fig:tokenizer" style="height: 300px;">
  <img src="./img/tf_tokenizer.jpg" style="height: 300px;" alt="Eingabepipeline mit Tokenizer, Embedding und Positional Encoding."/>
  <figcaption>Abbildung 2: Eingabepipeline eines Transformer-Netzwerks</figcaption>
</figure>
<br>


### Byte-Pair Encoding

Byte-Pair Encoding nutzt ein Vokabular mit einer festgelegten Länge. In unserer Implementation des Tokenizer nutzt er ein Vokabular von 8000 Einheiten. Das Vokabular wird folgendermaßen erstellt:

  1. Ein Text, der für die Erstellung des Vokabulars verwendet wird, wird in eine Sequenz von Buchstaben und Symbole zerlegt. Wortenden werden mit einem zusätzlichen Symbol kodiert.
  2. Alle Buchstaben und Symbole werden in das Vokabular aufgenommen.
  3. Nun wird das häufigste 2-Gramm, also zwei aufeinander folgende Symbole, gesucht, das im Text zu finden ist.
  4. Dieses wird ins Vokabular aufgenommen und im Text durch ein einzelnes Symbol ersetzt.
  5. Dieser Prozess wird nun wiederholt, bis die vorgegebene Länge des Vokabulars erreicht ist.

Dabei werden sowohl ganze Wörter ins Vokabular aufgenommen, wenn sie oft genug auftauchen (bespielweise werden die Worte "a", "the", "and" bei englischen Texte sicher aufgenommen), aber auch einzelne Wortteile wie z.B. "en##", "##ment" oder "##ed" werden in diesem Vokabular sicher vorkommen, um seltene Kombinationen wie "enablement" in die Wortteile "en##", "able" und "##ment" zerlgen zu können oder grammatikalische Formen wie "wanted" zu bilden.

In unserem Testbeispiel können Sie testen, wie Ihre Eingabe in Tokens getrennt und dann in numerische Werte umgewandelt wird, je nachdem, welche Position das Token in unserem Vokabular hat.
Wie Sie sehen, wird unser Bytepaar-Kodierungsvokabular durch ein [START]- und [END]-Token erweitert und enthält auch Elemente vom Typ 'abc##' oder '##abc'. Diese stellen eine Sequenz am Anfang oder Ende eines Wortes dar.

In [19]:
tokenizer = StoryTokenizer(reserved_tokens, vocab_path)

input_widget_tok = widgets.Text(
    value='Tokenizer test',
    description='Your input:',
    continuous_update=False,  
    layout = widgets.Layout(width='auto', margin='0px 0px 10px 0px')
)

button_widget_tok = widgets.Button(description='Run tokenizer on input',
                               layout = widgets.Layout(width='auto'))

output_widget_tok = widgets.Output(layout = widgets.Layout(width='auto'))


def tokenize(input_widget_tok):
    tokens = tokenizer.tokenize(input_widget_tok.value)                    # Erstellung der Tokens als Index für Vokabular
    lookup = tokenizer.lookup(tokens)                                      # Abrufen der Zeichenkette des Index im Vokabular                    
    
    return tokens, lookup
    
def on_button_click(b):
    with output_widget_tok:
        output_widget_tok.clear_output()                                                        
        tokens, lookup = tokenize(input_widget_tok)

        VisualWrapper.display_text('Tokens die aus der Eingabe mit Byte-Pair Encoding extrahiert werden:'.rjust(100) + ', '.join([token.decode('utf-8').rjust(10) for token in lookup.numpy()[0]])
                                   .replace(' ', '&nbsp;'))
        VisualWrapper.display_text('Ihre Positionsnummer im Alphabet des Byte-Pair Encoding Algorithmus:'.rjust(100) + ', '.join([str(token).rjust(10) for token in tokens.numpy()[0]])
                                   .replace(' ', '&nbsp;'))

button_widget_tok.on_click(on_button_click)

display(input_widget_tok, button_widget_tok, output_widget_tok)

Text(value='Tokenizer test', continuous_update=False, description='Your input:', layout=Layout(margin='0px 0px…

Button(description='Run tokenizer on input', layout=Layout(width='auto'), style=ButtonStyle())

Output(layout=Layout(width='auto'))

### Einbettung

Wo die Sequenzen die durch den Tokenizer entsteht in ihrer Länge, je nach Länge der Eingabe, aber auch durch die Eigenschaften des Byte-Pair-Encoding, variiert, ist es für Transformer notwendig eine gleichbleibendes Eingabeformat zu erhalten.

Dies wird dadurch gelöst, dass Padding Tokens zum Einsatz kommen, also Tokens, die keine Information codieren, sondern der Eingabe beigefügt werden, um die erforderliche Länge zu erreichen.

Außerdem wird die Ausgabe des Tokenizers mithilfe einer trainierbaren Einbettungsmatrix in das notwendige Format gebracht

Die Einbettung sorgt dafür, dass die Ausgabe des Tokenizer in einen Vektor der Modellgröße $d_{model}$ kodiert wird. Das heißt jedes Token, das zuvor durch eine Zahl kodiert wurde wird nun durch einen Vektor der Länge $d_{model}$ kodiert. 
Die Einbettung ist Teil der vom Modell gelernten Parameter (siehe <a href="#fig:embedding">Abbildung 5</a>) und somit nicht deterministisch gegeben. Welche Informationen aus der Ausgabe des Tokenizers wo gespeichert wird ist also nicht nachvollziehbar.
Vorstellen kann man sich aber, dass in jedem der $d_{model}$ Parametern einige der Informationen gespeichert werden, die für das jeweilige Token wichtig sind. Beispielsweise die Bedeutung des Tokens, seine grammatikalische Form, in welchem Kontext es benutzt wurde, steht es am Anfang eines Satzes oder am Ende, etc.


<figure id="fig:embedding" style="height: 300px;">
  <img src="./img/tf_embedding.jpg" style="height: 300px;" alt="Eingabepipeline mit Tokenizer, Embedding und Positional Encoding."/>
  <figcaption>Abbildung 4: Eingabepipeline eines Transformer-Netzwerks</figcaption>
</figure>
<br>


Wie eine solche Einbettung aussieht und wie sie sich verändert, wenn man beispielsweise neue Teile an den Satz anfügt können Sie in der nachfolgenden Simulation ausprobieren. Der Eingabetext wird erst vom Tokeniser in Tokens umgewandelt und dann durch die Einbettung in einen Tensor.

An jeder Position (vertikal dargestellt) ist dann die Einbettung des Tokens an dieser Stelle zu sehen (horizontal dargestellt). Die farbliche Kodierung stellt dabei das Zahlenspektrum dar, indem sich die Vektoreinträge bewegen.

In [16]:
class EmbeddingExample():

    def __init__(self) -> None:
        self.tokenizer = StoryTokenizer(reserved_tokens, vocab_path)

        self.input_widget = widgets.Text(
            value = 'Einbettung Test',
            description = 'Ihre Eingabe:',
            continuous_update=False,  # updates value only when you finish typing or hit "Enter"
            layout = widgets.Layout(width='auto', margin='0px 0px 10px 0px')
        )

        self.button_widget = widgets.Button(description='Einbettung erstellen',
                                    layout = widgets.Layout(width='auto'))

        self.output_widget = widgets.Output(layout = widgets.Layout(width='auto'))
        self.old_context = None

    def create_tokenized_embeddings(self):
        tokens = self.tokenizer.tokenize(self.input_widget.value)                                 # Tokenisierung der Eingabe
        tokens_all = tokens[tf.newaxis, :, :]                                                     # Hinzufügen einer weiteren Dimension
        input_without_eos = tokens[tf.newaxis, :, :-1]                                            # Auswahl der Tokens bis zum [END] Token
        token_input = self.tokenizer.detokenize(tokens_all)                                       # Nur zur Ausgabe Zwecken
        string_value = token_input.numpy()[0][0].decode('utf-8')                                  # Nur zur Ausgabe Zwecken
        lookup = tokenizer.lookup(input_without_eos)                                              # Nur zur Ausgabe Zwecken
        lookup = [item.decode('utf-8') for sublist in lookup.numpy()[0] for item in sublist]      # Nur zur Ausgabe Zwecken
        print("Wörter: ", string_value)
        print("Tokens: ", lookup)
        context = model.model.enc_embed(input_without_eos)                                        # Erstellung des Kontext Embedding 
        VisualWrapper.display_text('So sieht die Einbettung der Eingabe aus.')
        VisualWrapper.color_bar(context.to_tensor())
        if self.old_context is not None:
             padded_context, padded_old_context = self.pad_tensors(context, self.old_context)     # Erstellung des Padding Vektors der Eingaben
             VisualWrapper.display_text('So unterscheiden sich die alte und die neue Einbettung voneinander.')
             context_diff = padded_context - padded_old_context                                   # Berechnung der Unterschiede beider Vektoren
             VisualWrapper.color_bar(context_diff)

        self.old_context = context

    
    def on_button_click(self, b):
        with self.output_widget:
            self.output_widget.clear_output()  # clear the previous output
            VisualWrapper.reset_visualiser()
            self.create_tokenized_embeddings()
    
    def pad_tensors(self, ragged_tensor1, ragged_tensor2):
        """Funktion um die Tensoren der Eingabe auf die gleiche Länge zu transformieren"""
        tensor1 = ragged_tensor1.to_tensor()                                                     # Umwandlung in normalen Tensor
        tensor2 = ragged_tensor2.to_tensor()                                                     # Umwandlung in normalen Tensor

        shape1 = tf.shape(tensor1)
        shape2 = tf.shape(tensor2)

        target_shape = []

        for i in range(shape1.shape[0]):                                                         # Iterieren über die Dimensionen der Tensoren
            target_shape.append(tf.maximum(shape1[i], shape2[i]))                                # Die maximale Größe der Dimension wird an die Zielform angehängt.

        target_shape = tf.stack(target_shape)                                                    # Umwandlung der Zielform in einen Tensor


        paddings1 = []
        paddings2 = []

        for i in range(shape1.shape[0]):                                                         # Iterieren über die Dimensionen der Tensoren
            paddings1.append([0, target_shape[i] - shape1[i]])                                   # Auffüllung der Tensor auf maximale Länge 
            paddings2.append([0, target_shape[i] - shape2[i]])                                   # Auffüllung der Tensor auf maximale Länge 

        paddings1 = tf.stack(paddings1)                                                          # Konvertieren der Paddings in Tensoren
        paddings2 = tf.stack(paddings2)                                                          # Konvertieren der Paddings in Tensoren

        tensor1_padded = tf.pad(tensor1, paddings1)                                              # Tensoren an die Zielform anpassen
        tensor2_padded = tf.pad(tensor2, paddings2)                                              # Tensoren an die Zielform anpassen

        return tensor1_padded, tensor2_padded

emb_ex = EmbeddingExample()

VisualWrapper.display_text('Hier kannst du einen Text einbetten lassen. Wenn du die Eingabe veränderst wird außerdem gezeigt, wie sich die Einbettung geändert hat.')

emb_ex.button_widget.on_click(emb_ex.on_button_click)
display(emb_ex.input_widget, emb_ex.button_widget, emb_ex.output_widget)


HTML(value='<p style="font-size:18px; color:blue;">Hier kannst du einen Text einbetten lassen. Wenn du die Ein…

Text(value='Einbettung Test', continuous_update=False, description='Ihre Eingabe:', layout=Layout(margin='0px …

Button(description='Einbettung erstellen', layout=Layout(width='auto'), style=ButtonStyle())

Output(layout=Layout(width='auto'))

#### Erklärung des Beispiels
In diesen Beispiel wie ein Embedding grafisch aussieht. Das Embedding wird in der Funktion *create_tokenized_embeddings()* erstellt. Dazu wird zu erst der Eingabetext vom Tokenizer in Tokens unterteilt (Zeile 20). Die Tokens können Sie über der Grafik sehen. In Zeile 29 werden diese dann vom Transformer Modell in die Embeddings umgewandelt. 

Verwendet man zum Beispiel das Eingabebeispiel "Einbettung Test" und als zweites "Einbettung Test neu", sieht man im zweiten Bild wie die zusätzlichen Positionen dazu kommen. Die Visualisierung von "Einbettung Test" zeigt auf der y-Achse sieben Tokens. Bei dem Satz "Einbettung Test neu" kommen dann 3 Tokens dazu, diese sind "#n", "##e", "##u ". Ebenso kann man ein Muster bei dem Wertebereich erkennen. Der Bereich der Tiefe zwischen 0bis 256 bewegt sich hauptsächlich im Wertebereich -1 bis 2 und der Bereich von 257 bis 512 im Wertebereich 0 bis -3. Dies geht auf die Sinus-Funktion des Positional Encodings zurück, welche auf das Embedding angewendet wird. Diese hat für diese Bereiche stark unterschiedliche Werte. 

Die Werte in den ersten 256 Positionen können bestimmte Merkmale oder Eigenschaften der Wörter repräsentieren, während der Bereich der Positionen von 266 bis 512 andere Merkmale oder Eigenschaften widerspiegelt. Diese getrennte Darstellung ermöglicht dem Modell, komplexe Beziehungen und Muster in den Daten zu erfassen.




### Positionale Kodierung

Da in der Einbettung keine Informationen über die Position der verschiedenen Worte kodiert wird, muss diese manuell hinzugefügt werden. Hierfür benutzt die Transformerarchitektur für jede Position der Einbettung (also jedes enkodierte Wort) eine Sinuskurve mit anderer Frequenz und Phase. [1] Diese wird der Einbettung an der jeweiligen Stelle hinzugefügt.

Dadurch lassen sich die verschiedenen Worte sehr gut voneinander trennen. Die Idee dahinter ist, dass sich:
1. die grobe Position der Worte durch die langfrquenten Sinuskurven bestimmen lässt, da sie sich über die gesamte Länge der Eingabe nur allmählich verändert und die Werte der Einbettung durch diese insgesamt in eine bestimmte Richtung verschoben werden, z.B. die Worte im hinteren Teil der Eingabe größere Werte besitzen als im vorderen Teil der Eingabe,
2. die genaue Position durch die hochfrequentere Sinuskurven bestimmen lässt, da diese sich schon für benachbarte Vektoren unterscheidet und somit klar wird, welches Wort an welcher Stelle in der Einbettung kodiert wurde.

In der unten stehenden Simulation ist zu sehen, wie die positionale Kodierung beipielhaft für eine 2048 x 512 lange und tiefe Einbettung aussieht.

In [8]:
@interact(length=(2,2048,1), depth=(2,512,1))
def print_pos_enc(length, depth):
    VisualWrapper.color_bar(positional_encoding(length, depth))

interactive(children=(IntSlider(value=1025, description='length', max=2048, min=2), IntSlider(value=257, descr…

#### Erklärung des Beispiels
An diesem Beispiel sieht man den Effekt des Positional Encodings auf verschiedene lange bzw. tiefe Einbettungen. Wenn man mit den zwei Reglern spielt, sieht man, verschieden große Zooms auf den Effekt. Erhöht man den "length"-Regler sieht man die Auswirkungen, welche die Länge des Satzes betreffen. Die Werte die durch das Positional Encoding zu dem Vektor hinzugefügt werden sind dabei jedoch immer dieselben. Zum Bespiel wird an Stelle 100 immer derselbe Wert hinzugefügt, egal ob die maximale Länge des Satzes ("length") sich verändert. Dasselbe gilt auch für die Tiefe jedes einzelnen Vektors.


## Trainingsmethoden

### Dropout

Dropout [11] ist eine Methode, die während des Trainingsprozesses eines neuronalen Netzes genutzt wird, um zu verhindern, dass die gelernte Ausgabe eines Modells sich zu sehr auf einen einzelnen Prädikator stützt.
Dafür wird zwischen zwei Schritten desselben Modells, eine Dropout-Layer eingefügt. Diese setzt zufällig einige der vom ersten Modellteil generierten Ausgabe auf einen vordefinierten Wert (meistens -inf), um den nachfolgenden Schichten diese Information vorzuenthalten. Da diese Operation zufällig erfolgt, müssen die nachfolgenden Teile des Modells lernen ihre Ausgabe auch ohne diese Information zu erstellen. Somit lernt das Model seine Vorhersage auf eine möglichst breite Kombination an Merkmalen aufzubauen und man verhindert, dass Vorhersagen nur aufgrund eines einzigen Merkmals der vorherigen Ausgabe gemacht werden.

Ein gutes Beispiel ist das Ende eines Satzes vorherzusagen. In europäischen Sprachen wird ein Satz fast immer mit einem Punkt beendet, also ist es eine gute Strategie zu lernen, dass ein Satz druch einen Punkt beendet wird. Doch ein Modell, dass einen Punkt als einziges Merkmal eines Satzendes nutzt ist wenig robust, wenn man an falscher Stelle einen Punkt setzt oder ihn an einem Satzende druch ein anderes Zeichen ersetzt. Dabei gibt es auch andere Hinweise auf ein Satzende, wie z.B. das vorkommen eines Verbs in der deutschen Sprache oder von Ort und Zeitangaben im Englischen.

Um dem Modell keine Informationen vorzuenthalten, wenn es tatsächlich eingesetzt wird, ist Dropout immer so konzipiert, dass es zwar während des Trainings aktiv ist, aber danach abgeschalten wird, sodass während der Inferenzphase keine Informationen gelöscht werden.

Wie sich eine Ausgabe eines Teils eines Transformermodells unterscheiden kann, wenn man das Dropout anwendet können Sie in der nachfolgenden Simulation testen.

In [30]:
tokenizer_drop = StoryTokenizer(reserved_tokens, vocab_path)

input_widget_drop = widgets.Text(value = 'Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test',
                                 description = 'Ihre Eingabe:',
                                 continuous_update=False,  # updates value only when you finish typing or hit "Enter"
                                 layout = widgets.Layout(width='auto', margin='0px 0px 10px 0px')
                                 )

length_widget_drop = widgets.IntSlider(value=30,
                           min=2,
                           max=2048,
                           description='Länge des Tensors:',
                           continuous_update=False,  # updates value only when you finish typing or hit "Enter"
                           )
depth_widget_drop = widgets.IntSlider(value=512,
                          min=2,
                          max=512,
                          description='Tiefe des Tensors:',
                          continuous_update=False,  # updates value only when you finish typing or hit "Enter"
                          )
dropout_widget = widgets.FloatSlider(value=0.1,
                              min=0,
                              max=0.9,
                              step=0.1,
                              description='Dropoutrate:',
                              continuous_update=False,  # updates value only when you finish typing or hit "Enter"
                              )



def dropout_function(length, depth, dropout, input):

    # Erstellung der Dropout Layer
    dropout_layer = layers.Dropout(dropout)                                                         # Anwendung des Dropout auf die Layer
    one_tensor = tf.ones([length, depth])                                                           # Erstellung eines Arrays aus 1-en
    dropout_tensor = dropout_layer(one_tensor, training=True)                                       # Anwendung des Dropout auf die Layer

    # Erstellung der Kontext Layer
    tokens = tokenizer_drop.tokenize(input)                                                         # Tokenisierung des Inputs
    input_without_eos = tokens[tf.newaxis, :, 1:-1]                                                 # Auswahl der Tokens bis zum [END] Token
    context = model.model.enc_embed(input_without_eos)                                              # Erstellung des Embedding durch das Modell
    context_drop = dropout_layer(context, training=True)                                            # Anwendung des Dropout auf das Embedding

    return dropout_tensor, context_drop, context

def out(length, depth, dropout, input):
    VisualWrapper.reset_visualiser()                                                   
    dropout_tensor, context_drop, context = dropout_function(length, depth, dropout, input)
    VisualWrapper.color_bar(dropout_tensor)
                               
    VisualWrapper.color_bar(context.to_tensor())                     
    VisualWrapper.color_bar(context_drop.to_tensor())
    

output_widget_dropout = widgets.interactive_output(out,
                                                   {'length': length_widget_drop, 'depth': depth_widget_drop, 'dropout': dropout_widget, 'input': input_widget_drop}, 
                                                   )

display(length_widget_drop, depth_widget_drop, dropout_widget, input_widget_drop, output_widget_dropout)


IntSlider(value=30, continuous_update=False, description='Länge des Tensors:', max=2048, min=2)

IntSlider(value=512, continuous_update=False, description='Tiefe des Tensors:', max=512, min=2)

FloatSlider(value=0.1, continuous_update=False, description='Dropoutrate:', max=0.9)

Text(value='Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Tes…

Output()

#### Erklärung des Beispiels
In diesem Beispiel wird der Effekt von Dropout auf die Layer und das Embedding dargestellt. Dafür werden in der Funktion *dropout_function()* jeweils Dropout auf die Layer und auf das Embedding angewendet. Dafür wird in Zeile 36 und 42 jeweils die jeweilige Transformation auf den beiden Objekten angewendet. Damit wird also ein gewisser Teil, welche mit dem Parameter "Dropoutrate" bestimmt wird, der Werte Layer bzw. des Embeddings den weiteren Verarbeitungsschritten vorenthalten. Dieser Parameter ist ein prozentualer Wert, d.h. bei einem Wert von 0.2 werden 20% der Werte vorenthalten. 

Für das Beispiel können dieses Mal die Länge des Tensors (Eingabe) und die Tiefe des Tensors bestimmt werden. Ebenso kann die Dropoutrate verändert werden. In der ersten Grafik sieht man welche Werte in einem uniformen Vektor vom Dropout verändert werden. Das bedeutet, welche der Werte in der Tiefe des Tensors ausgeblendet werden. Der Wert ist entspricht dabei einer Prozentzahl bei 0.4 werden somit 40 Prozent der Werte ausgeblendet bzw. auf den festgesetzten Wert gesetzt.
In den beiden darauffolgenden Grafiken wird das Dropout auf den im Textfeld eingegebenen Beispieltext angewandt, nachdem er durch den Tokenizer und die Einbettung in Vektorform gebracht wurde. Die erste Grafik zeigt den vollständigen Vektor und die zweite Grafik den Vektor, der vom Dropout verändert wurde. Hier sieht man die ausgelassenen Positionen sehr gut. Mit dem Erhöhen der Dropout Rate, werden diese mehr. Außerdem kann man erkennen, dass sich, wenn man den Dropout erhöht, der Wertebereich ebenfalls ausweitet. Erklärbar ist dies dadurch, dass Merkmale, vor allem prägnante Merkmale, sich durch das weitere Training verstärken bzw. abschwächen. Die Dropoutrate kann jedoch nicht beliebig erhöht werden bei gleichzeitigem Erhalt der Güte der Klassifikation.
 


### Normalisierung

Normalisierung ist eine Technik, die von [20] eingeführt wurde. In tiefen neuronalen Netzen, mit nicht-linearen Aktivierungsfunktionen wie der Sigmoid-Funktion

$$g(x) = 1/(1+exp(-x))$$

trainiert werden gilt, dass $g’(x) -> 0 für |x|-> inf$. 

Das führt dazu, dass diese Funktionen in einen Bereich geraten können, in der der Gradient für Stochastic Gradient Descent (SGD) minimal wird und man spricht vom Vanishing Gradient Problem.

In neuronalen Netzen ist hierbei das Problem, dass eine Layer $z = g(Wx + b)$ mit der Sigmoid-Funktion g versucht den Output des gesamten vorherigen Netzes x zu gewichten. Dabei hängen sowohl W als auch b von x ab. Da sich während des Trainings alle Layers des Netzes fortwährend aktualisieren, ändert sich auch der Input x fortwährend. 
Dieser Effekt wird von [20] Internal Covariate Shift genannt. 

Je tiefer das neuronale Netz, umso größer sind diese Veränderungen, da es mehr Schichten gibt, die sich verändern können. Die Tiefe einer neuronalen Netzes erhöht also die Wahrscheinlichkeit das ein Vanishing Gradient Problem auftritt und ein effektives Training stoppt.

Transformer wie sie [1] beschrieben nutzen hierfür Layer Normalisation ein Normalisierungsalgorithmus den [21] entwickelt hat.

<span style="color:red">Interactive Application - In- Output of LayerNorm Comparison funktioniert nicht richtig.</span>



In [10]:
VisualWrapper.reset_visualiser()
tokenizer_norm = StoryTokenizer(reserved_tokens, vocab_path)

input="Test"

tensor_input = tf.convert_to_tensor(input)
if len(tensor_input.shape) == 0:
    tensor_input = tensor_input[tf.newaxis]

tokenized_input = tokenizer_norm.tokenize(tensor_input).to_tensor()                             # Anwendung eines Tokenizers mit Normalisierung
input_without_eos = tokenized_input[:, :-1]
context = model.model.encode(input_without_eos, None)

VisualWrapper.visualize_data(id='layer')



### Residuale Verbindung

Die Idee für das Nutzen von Residualen Verbindungen kommt von [22]. Die Autoren stellten fest, dass bei neuronalen Netzen sowohl die Genauigkeit während des Trainings als auch die Genauigkeit auf dem Testdatensatz mit zunehmender Tiefe schlechter wird.

Da durch Normalisierung bereits sichergestellt ist, dass Vanishing Gradients kein Problem darstellen, scheitert die Optimierung der neuronalen Netze aus anderen Gründen.
Einer der Gründe hierfür liegt vermutlich darin, dass die tieferen Schichten eines Modells zu Beginn des Trainings sehr viel stärker zur Ausgabe beitragen, als die vorhergehenden Schichten. Sie werden somit zuerst trainiert. Die weniger tiefen Schichten werden erst ausreichend trainiert, wenn in den tiefen Schichten keine Optimierung mehr möglich ist.

Eine Lösung hierfür sind die residualen Verbindungen.
Residuale Verbindungen ersetzen eine Schicht F(x) durch ihre residuale Verbindung

\
$$H(x) = F(x) + x.$$

<br>
In das Ergebnis von H(x) geht also sowohl der Output, als auch der Input von F direkt mit ein. Wendet man dieses Prinzip auf die Schichten tiefer neuronaler Netze an, sorgt das dafür, dass gleich zu Beginn, der Output der wenig tiefen Schichten relevant in den Output des gesamten Netzes einfließt, denn es gilt für das gesamte Netz N:

\
$$N(x) = H_n(H_n-1(x)) + H_n-1(x) = H_n(H_n-1(x)) + H_n-1(H_n-2(x)) + … + H_2(H_1(x)) + H_1(x)$$

<br>
Wie man in <a href="#fig:input">Abbildung 1</a> sehen kann, haben alle Aufmerksamkeitsmodule, sowie alle Feed-Forward Schichten in einem Transformermodell eine residuale Verbindung.

In der nachfolgenden Simulation können Sie sehen, wie sich die Ausgabe einer neuronalen Schicht verändert, wenn man ihr eine residuale Verbindung beifügt. Der Effekt auf den Trainingsprozess lässt sich dabei natürlich nur schwer darstellen.


In [11]:
# Interaktiver Vergleich mit/ohne residuale Verbindung. Evtl. wäre es auch spannend den Trainingsprozess hier zu starten und Veränderungen dabei aufzuzeigen. Dies geht aber vermutlich interaktiv nicht, da es zu lange dauert und zu viele Ressourcen benötigt. Man könnte allerdings eine Aufzeichnung der zugehörigen Daten machen. Dies zu implementieren ist aber gerade nicht machbar.



## Modellschichten

### Aufmerksamkeit

Die Neuerung von Transformern im Vergleich zu vorangegangenen Lösungen für Neuronalen Maschinellen Übersetzen (NMT) ist es, allein auf die Aufmerksamkeit als Mechanismus für das Verarbeiten von Sprache zu setzen. Aufmerksamkeit wurde auch vorher schon von [15] zur Verbesserung von RNNs zur Übersetzung von Texten verwendet.

Der Aufmerksamkeitsmechanismus, wie ihn [1] beschreiben, orientiert sich dabei an der Idee einer Suchanfrage des Ausgabetextes and sich selbst, bzw. den Eingabetext. Die Aufmerksamkeitsschicht bekommt dabei zwei oder eigentlich drei Eingaben: 

1. den Query (Q), 
2. den Key (K),
3. den Value (V).

In der Praxis erhalten aber Key und Value in Transformern immer dieselbe Eingabe und häufig sind Query, Key und Value sogar identisch. Aus welcher Quelle Query, Key und Value kommen unterscheidet unterschiedliche Formen von Aufmerksamkeit. So nennen wir Selbstaufmerksamkeit denjenigen Fall indem Q=K=V gilt und Kreuzaufmerksamkeit denjenigen Fall indem der Query aus der Ausgabe des Encoder besteht und Key und Value beide aus der Ausgabe eines Decoderblocks stammen.

Um zu erklären, wie Aufmerksamkeit funktioniert, sollten wir aber zunächst davon ausgehen, dass Query-, Key- und Value-Eingabe verschieden sind. Ich schreibe bewusst von der Eingabe, da in jeder Aufmerksamkeitsschicht zunächst Query-, Key- und Value-Eingabe Q', K', V' mit Hilfe von gewichteten Matrix W^Q, W^K und W^V in Query, Key und Value 

$$Q=Q′×W^Q$$
$$K=K′×W^K$$
$$V=V′×W^V$$ 

umgewandelt werden. Diese gewichteten Matrizen W^Q, W^K, W^V sind die trainierbaren Gewichte der Aufmerksamkeitsschicht. Alle nachfolgenden Prozesse sind deterministisch. Das bedeutet, dass diese Umwandlungprozesse festelegen, welches Ergebnis die Aufmerksamkeitsschicht liefert.
Betrachten wir aber, was passiert, wenn Query, Key und Value durch diese Matrizen gewichtet wurden. Eine grafisch aufbereitete Erklärung dessen, was ich beschreibe findet sich übrigens bei [9].

#### Aufmerksamkeitsfunktion

Die Aufmerksamkeitsfunktion bekommt die Eingaben Q, K, V aus der vorher beschriebenen Gewichtung und lautet:
<br>
<br>
$$Attention(Q, K, V)=softmax(QK^T/sqr(d_k))V$$
<br>
Es wird also zuerst das Kreuzprodukt aus Q und K gebildet. Dieses Produkt wird mit sqr(*d_k*) skaliert, weshalb ist in folgendem [Kaptiel](#skalierung-mit-sqrd_k) nachzulesen, und auf dieses Ergebnis wird dann die Softmax-Funktion sigma(x) angewandt.
Diese Funktion lässt sich am besten positionsweise beschreiben
<br>
<br>
$$sigma(x)_i = exp(x_i)/sum_j=1^n(exp(x_j)) für i=1,...,n.$$
<br>
Nennen wir also
<br>
$$softmax(QK^T/sqr(d_k)) = Score(Q,K),$$
<br>
dann ist 
<br>
$$Attention(Q, K, V) = Score(Q,K)*V $$
<br>
eine Funktion, die $V$ mit einem Vektor multipliziert, wobei für den Vektor gilt $|Score(Q,K)| = 1$.

$Score(Q,K$) ist also ein Vektor exakt der Länge von $V$, der angibt, mit welchem Anteil jeder Eintrag von $V$ in den Ausgabevektor $Attention(Q,K,V)$ eingehen soll. Dabei summiert sich $Score(Q,K)$ zu 1, ist also tatsächlich eine Gewichtung der Einträge von $V$.

Da mit $Score(Q,K)$ nun also eine Gewichtung besteht, wie stark die Ausgabe $Attention(Q,K,V)_i$ von $V_j$ abhängt, kann man dieses Verhältnis als Matrix grafisch darstellen. Dies geschieht in der Simulation unten, bei der für den vorgegebenen Satz eine Aufmerksamkeitsgewichtung aus einer der Aufmerksamkeitsschichten dargestellt wird. Der Eintrag in der $i-ten$ Zeile und $j-ten$ Spalte gibt dabei den einfluss von $V_j$ auf $Attention(Q,K,V)_i$ an.


In [12]:
tokenizer_attn = StoryTokenizer(reserved_tokens, vocab_path)
attn_model = WordComplete(StoryTokenizer(reserved_tokens, vocab_path), model.model, max_length=32)

input_widget_attn = widgets.Text(
    value='A longer test sentence is more interesting, as it allows to see dependencies more clearly.',
    description='Ihre Eingabe:',
    continuous_update=False,  # updates value only when you finish typing or hit "Enter"
    layout = widgets.Layout(width='auto', margin='0px 0px 10px 0px')
)

button_widget_attn = widgets.Button(description='Aufmerksamkeit berechnen',
                               layout = widgets.Layout(width='auto'))

output_widget_attn = widgets.Output(layout = widgets.Layout(width='auto'))

def create_tokenized_embeddings():
        tensor_input = tf.convert_to_tensor(input_widget_attn.value)                # Umwandelung des Textinputs in ein TensorFlow-Tensor 
        output_array = tf.TensorArray(dtype=tf.int64, size=0, dynamic_size=True)    # Erstellung eines leeren TensorArrays für die spätere Ausgabe
        if len(tensor_input.shape) == 0:                                            # Überprüft, ob der Eingabetensor im korrekten Format ist                                     
            tensor_input = tensor_input[tf.newaxis]                                 # Falls nicht, wird eine Dimension hinzufügt 

    
        tokenized_input = tokenizer.tokenize(tensor_input).to_tensor()              # Umwandlung des Textinputs in Tokens und anschließend in einen Tensor
        input_without_eos = tokenized_input[:, :-1]
        context = transformer.encode(input_without_eos, None)                       # Erstellung der Kontext-Vektoren vom Transformer-Modell

        # Write the input tokens (excluding the last one) to the output array
        for i, value in enumerate(tokenized_input[0][:-1]):
            output_array = output_array.write(i, value)

        dec_input = output_array.concat()[tf.newaxis]

        dec_out = transformer.decode(context, None, dec_input, None)

def on_button_click(b):
    with output_widget_attn:
        #VisualWrapper.n_vis_layers_per_class['MultiHeadedAttention'] = 6
        #output_widget_attn.clear_output()  # clear the previous output
        create_tokenized_embeddings()
        VisualWrapper.visualize_data(id='attention')
        #VisualWrapper.n_vis_layers_per_class['MultiHeadedAttention'] = 1
            

button_widget_attn.on_click(on_button_click)

display(input_widget_attn, button_widget_attn, output_widget_attn)

Text(value='A longer test sentence is more interesting, as it allows to see dependencies more clearly.', conti…

Button(description='Aufmerksamkeit berechnen', layout=Layout(width='auto'), style=ButtonStyle())

Output(layout=Layout(width='auto'))


#### Multi-headed Aufmerksamkeit

In der Praxis hat sich bewährt parallel mehrere dieser Aufmerksamkeitsmechanismen durchzuführen. Dazu werden zu Beginn h gewichteten Matrizen 

$$W_i^Q, W_i^K, W_i^V i= 1,...,h$$ 

eingeführt. Diese erzeugen also h verschiedene Matrixtripel 

$$Q_i, K_i, V_i i = 1,...,h$$

und somit ergeben sich h verschiedene Ausgaben 

$$H_i = Attention(Q_i, K_i, V_i) i=1,...,h.$$

Diese werden nun zu einer einzigen Ausgabe zusammengeführt, indem wir 

$$MultiHeadAttention(Q_in,K_in,V_in) = Concat(H_1,..., H_h)W^O$$

berechnen. Dabei ist $Concat(A_1,...,A_n)$ das hintereinanderschreiben mehrerer Matrizen und $W^O$ eine weitere trainierbare Matrix, die die verschiedenen Ausgaben $H_1,..., H,h$ gewichtet.
Deshalb sehen wir in der obigen Ausgabe auch x verschiedene Matrizen, die $Score(Q_i,K_i)$ darstellen.


### Maskieren

Das Maskieren des Inputs ist eine wichtige funktionale Komponente der Transformerarchitektur. Beim Maskieren handelt es sich in Wirklichkeit um zwei Mechanismen, die zwar dieselbe Funktionsweise besitzen, aber sehr unterschiedliche Aufgaben in der Architektur besitzen. Einerseits das Subsequent Masking und das Padding Masking. Das Padding Masking stellt jediglich sicher, dass nur Positionen mit Inhalt vom Transformer verarbeitet werden, während das Subsequent Masking dafür sorgt, dass der Decoder des Transformers autoregressiv ist. Das bedeutet, bei einer Vorhersage für eine Ausgabe an der Position $i$, soll das Modell nur Informationen aus den Eingabepositionen $1,…,i-1$ nutzen.


#### Padding Masking

Das Padding Masking ist notwendig, da Transformer sequentielle Daten so verarbeiten, als ob sie eine fixe Länge $d_{model}$ hätten. Das geschieht deshalb, weil Transformer so lernen können jede Position der Ausgabelänge $d_{model}$ parallel vorherzusagen. 

Um während des Trainings auch Daten mit einer Länge größer oder kleiner $d_{model}$ zu nutzen, werden längere Sequenzen abgeschnitten und kürzere mit Nullen aufgefüllt. Diese Nullen müssen dann mit Hilfe von Padding Masking für das Training irrelevant gemacht werden. Das geschieht indem man alle Positionen die eine Null enthalten für das Modell auf $-inf$ setzt, sodass sie beim Gradient Descent nicht berücksichtigt werden.

#### Subsequent Masking

Das Subsequent Masking benutzt die gleich Technik des Werte auf $-inf$ setzen. Dabei wird aber nicht das mit irrelevanten Informationen angefüllte Ende des Eingabevektors auf Null gesetzt, sondern es werden diejenige Werte von $Score(Q, V)$ auf $-inf$ gesetzt, die einen Wert $V_j$, für die Ausgabe $i$ $Attention(Q,K,V)_i$ gewichten würden, obwohl $j>i$ gilt. Also es wird beschränkt, dass $Score(Q, V)$ nur diejenigen Werte von $V$ einbeziehen darf, die bei der Vorhersage für den i-ten Wert schon bekannt sind.

Dass das relevant ist liegt daran, dass beim Training von Transformern ja der komplette Input gleichzeitig verwertet wird. Ein Satz wird als ganzes vom Transformer verarbeitet und er erzeugt eine Vorhersage, welches Token an einer bestimmten Position in diesem Satz vorkommt. Dabei erhält er in der Eingabe aber schon die Information, welches Token an dieser Position tatsächlich steht, da er ja den kompletten Satz als Eingabe bekommen hat. Dies muss nun also innerhalb des Modells mit einer Maskierung wieder rückgängig gemacht.


#### Verschiedene Aufmerksamkeitstypen

In der Architektur werden verschiedene Aufmerksamkeitstypen unterschieden. Es gibt dabei zwei Variablen die beeinflussen, welche Art von Aufmerksamkeit wir verwenden. Die erste Variable ist, woher die Eingaben $Q', K' und V'$ kommen. Die zweite Variable ist die Maskierung, die wir auf $Score(Q,K)$ anwenden.

##### Selbst-Aufmerksamkeit

Die Aufmerksamkeit nennen wir Selbst-Aufmerksamkeit, wenn gilt 

$$Q'=K'=V'.$$ 

Wenn sich $Score(Q,K)$ also bildlich gesprochen daraus ergibt, welche Aufmerksamkeit jede Position der Eingabe auf eine andere Position derselben Eingabe legt und diese Aufmerksamkeit auf die Eingabe selbst angewandt wird.

##### Kreuz-Aufmerksamkeit

Wie nennen die Aufmerksamkeit Kreuz-Aufmerksamkeit, wenn gilt 

$$K' = V'$$

aber $Q'$ von diesen beiden Werten verschieden ist.
Wenn sich $Score(Q,K)$ also daraus ergibt, welche Aufmerksamkeit jede Position einer Eingabe $Q'$ auf die Positionen einer zweiten Eingabe $K'$ gibt und dieser Aufmerksamkeitsscore auf die zweite Eingabe angewandt wird.

Dies ist zum Beispiel in Transformern der Fall, wenn $Q'$ sich aus der Ausgabe des Encoder ergibt und $K' = V'$ ein Zwischenergebnis des Decoders ist.

##### Maskierte Aufmerksamkeit

Ein Fall von maskierter Aufmerksamkeit liegt dann vor, wenn bestimmte Werte von $Score(Q,K)$ maskiert werden. Das ist zum Beispiel beim Subsequent Masking der Fall, hier wird $Score(Q,K)i,j = -inf$ gesetzt für alle Einträge $j>i$. Dadurch wird verhindert, dass die Ausgabe $Attention(Q,K,V)_i$ sich auf die Werte $V_j, j>i $ stützt. Zum Beispiel wird während des Trainings im Decoder dadurch verhindert, dass das Model lernt Informationen aus den zukünftigen Einträgen $V_j, j>i$ zu benutzen, um $V_i$ vorherzusagen. Man sieht gut in der Darstellung von $Score(Q,K)$, dass die Werte für $j>i$ dadurch meistens nahe bei $0$ liegen.


### Skalierung mit sqr(d_k)

Dasselbe Problem des Vanishing Gradient könnte innerhalb der Aufmerksamkeitsfunktion auftauchen, da hier $softmax(QK^T)V$ berechnet wird, das Skalarprodukt $QK^T$ mit $d_{model}$ skaliert und die Softmaxfunktion
    
$$sigma(z)_i = exp(z_i)/sum_j=1^N(exp(z_j)) für j=1,...,N $$

somit leicht in einen saturierten Bereich mit extrem kleinen Gradienten gerät. Deshalb wird $QK^T$ mit $sqr(d_{model})$ skaliert: $softmax(QK^T/sqr(d_{model}))$, um die Skalierung mit $d_{model}$ zu minimieren.


In [13]:
# Interaktive Anwendung vgl. eines Tensors vor und nach dem Maskieren

Zuletzt findet sich hier nun noch eine Simulation des kompletten Inferenzvorgangs innerhalb eines Transformermodells. In dieser Simulation kann man noch einmal alle vorher beschriebenen Schritte nachvollziehen.

In [14]:
inference_model = WordComplete(StoryTokenizer(reserved_tokens, vocab_path), model.model, max_length=32)

input_widget_inf = widgets.Text(
    value='Test sentence',
    description='Your input:',
    continuous_update=False,  
    layout = widgets.Layout(width='auto', margin='0px 0px 10px 0px')
)

button_widget_inf = widgets.Button(description='Run interactive inference',
                               layout = widgets.Layout(width='auto'))

output_widget_inf = widgets.Output(layout = widgets.Layout(width='auto'))

def on_button_click(b):
    with output_widget_inf:
        output_widget_inf.clear_output()  
        inference_model(input_widget_inf.value, interactive=True) 
        inference_model.print_results(visualisation=True)

button_widget_inf.on_click(on_button_click)

display(input_widget_inf, button_widget_inf, output_widget_inf)


Text(value='Test sentence', continuous_update=False, description='Your input:', layout=Layout(margin='0px 0px …

Button(description='Run interactive inference', layout=Layout(width='auto'), style=ButtonStyle())

Output(layout=Layout(width='auto'))

# Bibliography
[1] Vaswani, A. et al. Attention Is All You Need. Preprint at http://arxiv.org/abs/1706.03762 (2017).

[2] Hochreiter, S. & Schmidhuber, J. Long Short-Term Memory. Neural Computation 9, 1735–1780 (1997).

[3] Bengio, Y., Simard, P. & Frasconi, P. Learning long-term dependencies with gradient descent is difficult. IEEE Transactions on Neural Networks 5, 157–166 (1994).

[4] Cho, K., van Merrienboer, B., Bahdanau, D. & Bengio, Y. On the Properties of Neural Machine Translation: Encoder–Decoder Approaches. in Proceedings of SSST-8, Eighth Workshop on Syntax, Semantics and Structure in Statistical Translation 103–111 (Association for Computational Linguistics, 2014). doi:10.3115/v1/W14-4012.

[6] Kaplan, J. et al. Scaling Laws for Neural Language Models. Preprint at http://arxiv.org/abs/2001.08361 (2020).

[5] Goodfellow, I., Bengio, Y. & Courville, A. Deep learning. (The MIT Press, 2016).

[7] Radford, A. et al. Language Models are Unsupervised Multitask Learners. (2019).

[8] Devlin, J., Chang, M.-W., Lee, K. & Toutanova, K. BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding. Preprint at http://arxiv.org/abs/1810.04805 (2019).

[9] Alammar, J. The Illustrated Transformer – Jay Alammar – Visualizing machine learning one concept at a time. https://jalammar.github.io/illustrated-transformer/ (2018).

[10] Encoder-Decoder. Understanding The Model Architecture | by Naoki | Medium. https://naokishibuya.medium.com/transformers-encoder-decoder-434603d19e1.

[11] Srivastava, N., Hinton, G., Krizhevsky, A., Sutskever, I. & Salakhutdinov, R. Dropout: A Simple Way to Prevent Neural Networks from Overﬁtting. (2014).

[12] He, K., Zhang, X., Ren, S. & Sun, J. Deep Residual Learning for Image Recognition. Preprint at http://arxiv.org/abs/1512.03385 (2015).

[13] Gage, P. A New Algorithm for Data Compression. (1994).

[14] Cho, K. et al. Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation. Preprint at http://arxiv.org/abs/1406.1078 (2014).

[15] Bahdanau, D., Cho, K. & Bengio, Y. Neural Machine Translation by Jointly Learning to Align and Translate. Preprint at http://arxiv.org/abs/1409.0473 (2016).

[16] OpenAI. GPT-4 Technical Report. Preprint at http://arxiv.org/abs/2303.08774 (2023).

[17] Grefenstette, G. & Tapanainen, P. What is a word, What is a sentence? Problems of Tokenization.

[18] Lin, Z. et al. A Structured Self-attentive Sentence Embedding. Preprint at http://arxiv.org/abs/1703.03130 (2017).

[19] Schmidt, R. M. Recurrent Neural Networks (RNNs): A gentle Introduction and Overview. Preprint at http://arxiv.org/abs/1912.05911 (2019).

[20] Ioffe, S. & Szegedy, C. Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift. Preprint at http://arxiv.org/abs/1502.03167 (2015).

[21] Ba, J. L., Kiros, J. R. & Hinton, G. E. Layer Normalization. Preprint at http://arxiv.org/abs/1607.06450 (2016).

[22] He, K., Zhang, X., Ren, S. & Sun, J. Deep Residual Learning for Image Recognition. Preprint at http://arxiv.org/abs/1512.03385 (2015).
